In [25]:
import pandas as pd
import numpy as np
import os
from scipy.signal import welch

from tqdm import tqdm
from datetime import datetime
import shutil
from geopy.distance import geodesic

In [26]:
def movement_ratios(df):
    try:
        df['time'] = pd.to_datetime(df['time'])
        total_time = (df['time'].iloc[-1] - df['time'].iloc[0]).total_seconds()

        active_time = (df['speed'] > 0).sum() / 100  # 10Hz 采样，每 10 个点代表 1 秒


        movement_ratio = active_time / total_time if total_time > 0 else 0
        return movement_ratio
    except:
        return np.nan
#movement_ratios(dfs)

0.0

In [27]:
def heart(df):
    try:
        df = df[df['speed'] > 0].copy()


        df['heart_rate_diff'] = df['heart_rate'].diff()

        result = df.agg(
            mean_heart_rate=('heart_rate', 'mean'),  
            max_heart_rate=('heart_rate', 'max'),     
            min_heart_rate=('heart_rate', 'min'),     
            sdnn=('heart_rate', 'std'),              
            rmssd=('heart_rate_diff', lambda x: np.sqrt(np.mean(x.dropna()**2))) 
        )
        return result['heart_rate']['mean_heart_rate'], result['heart_rate']['max_heart_rate'], result['heart_rate']['min_heart_rate'], result['heart_rate']['sdnn'], 
    except:
        return np.nan, np.nan, np.nan, np.nan
#heart(dfs)

/tmp/ipykernel_2093610/3540480073.py:9: FutureWarning: using <function heart.<locals>.<lambda> at 0x7f9e1432fb50> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  result = df.agg(


(nan, nan, nan, nan)

In [28]:
def haversine_distance(lat1, lon1, lat2, lon2):
   # """计算两点间球面距离（单位：米）"""
    return geodesic((lat1, lon1), (lat2, lon2)).meters
def total_distance(df):
    try:
        df = df[df['speed'] > 0].copy()
        df['prev_lat'] = df['lat'].shift()
        df['prev_lon'] = df['lon'].shift()

        df['distance'] = df.apply(lambda row: haversine_distance(row['prev_lat'], row['prev_lon'], row['lat'], row['lon'])
                                  if pd.notnull(row['prev_lat']) else 0, axis=1)

        total_distance = df['distance'].sum()  
        return total_distance
    except:
        return np.nan
#total_distance(dfs)

0.0

In [29]:
def speeds(df):
    try:
        df = df[df['speed'] > 0].copy()
        mean_speed = df['speed'].mean()
        max_speed = df['speed'].max()
        var_speed = df['speed'].var()
        return mean_speed, max_speed, var_speed
    except:
        return np.nan, np.nan, np.nan
#speeds(dfs)

(nan, nan, nan)

In [30]:
def accelerations(df):
    try:
        df = df[df['speed'] > 0].copy()
        mean_acceleration = df['inst_acc_impulse'].mean()
        max_acceleration = df['inst_acc_impulse'].max()
        var_acceleration = df['inst_acc_impulse'].var()
        shock_count = (df['inst_acc_impulse'] > 2).sum()
        return mean_acceleration, max_acceleration, var_acceleration, shock_count
    except:
        return np.nan, np.nan, np.nan, np.nan
#accelerations(dfs)

(nan, nan, nan, 0)

In [31]:
def accel_vector(df):
    try:
        df = df[df['speed'] > 0].copy()
        df['accel_vector'] = np.sqrt(df['accl_x']**2 + df['accl_y']**2 + df['accl_z']**2)
        mean_accel_vector = df['accel_vector'].mean()
        var_accel_vector = df['accel_vector'].var()
        jump_count = ((df['accl_z'] > 1.5) & (df['accl_z'] < 3)).sum()
        return mean_accel_vector, var_accel_vector, jump_count
    except:
        return np.nan, np.nan, np.nan
#accel_vector(dfs)

(nan, nan, 0)

In [32]:
import numpy as np
def gyro_vector(df):
    try:
        df = df[df['speed'] > 0].copy()
        df['gyro_vector'] = np.sqrt(df['gyro_x']**2 + df['gyro_y']**2 + df['gyro_z']**2)
        mean_gyro_vector = df['gyro_vector'].mean()
        var_gyro_vector = df['gyro_vector'].var()
        return mean_gyro_vector, var_gyro_vector
    except:
        return np.nan, np.nan
#gyro_vector(dfs)

(nan, nan)

In [36]:
import os
import pandas as pd
from datetime import datetime

data_folder = "datas"

# 目标运动员 ID 和队伍名称
athlete_ids = os.listdir('acwr/')
existing = os.listdir('timeseries/')
athlete_ids = [i for i in athlete_ids if i.endswith('.csv') and i not in existing]
#len(athlete_ids)

0

In [ ]:
all_files = os.listdir(data_folder)

for athlete_file in tqdm(athlete_ids):
    movement_ratio = []
    dates = []
    mean_speeds, max_speeds, var_speeds = [], [], []
    mean_hrs, max_hrs, min_hrs, sdnn_hrs = [], [], [], []
    total_distances = []
    mean_accelerations, max_accelerations, var_accelerations, shock_counts = [], [], [], []
    mean_gyro_vectors, var_gyro_vectors = [], []
    fea_df = pd.DataFrame()
    athlete_id = os.path.splitext(athlete_file)[0]
    athlete_files = []
    for file in all_files:
        date = file.split('.')[0].replace(f'-{athlete_id}', '')
        if file.endswith(".parquet") and athlete_id in file:
            try:
                df = pd.read_parquet(os.path.join(data_folder, file))
                if len(df)>20000:
                    df['time'] = pd.to_datetime(df['time'])
                    movement_ratio.append(movement_ratios(df))
                    mean_hrs.append(heart(df)[0])
                    max_hrs.append(heart(df)[1])
                    min_hrs.append(heart(df)[2])
                    sdnn_hrs.append(heart(df)[3])
                    total_distances.append(total_distance(df))
                    mean_speeds.append(speeds(df)[0])
                    max_speeds.append(speeds(df)[1])
                    var_speeds.append(speeds(df)[2])
                    mean_accelerations.append(accelerations(df)[0])
                    max_accelerations.append(accelerations(df)[1])
                    var_accelerations.append(accelerations(df)[2])
                    shock_counts.append(accelerations(df)[3])
                    mean_gyro_vectors.append(gyro_vector(df)[0])
                    var_gyro_vectors.append(gyro_vector(df)[1])
                    dates.append(date)
            except:
                print(file)
                pass
    fea_df = pd.DataFrame({
        "Date": dates,
        "Movement_Ratio": movement_ratio,
        "Mean_HR": mean_hrs,
        "Max_HR": max_hrs,
        "Min_HR": min_hrs,
        "SDNN_HR": sdnn_hrs,
        "Total_Distance": total_distances,
        "Mean_Speed": mean_speeds,
        "Max_Speed": max_speeds,
        "Var_Speed": var_speeds,
        "Mean_Acceleration": mean_accelerations,
        "Max_Acceleration": max_accelerations,
        "Var_Acceleration": var_accelerations,
        "Shock_Count": shock_counts,
        "Mean_Gyro_Vector": mean_gyro_vectors,
        "Var_Gyro_Vector": var_gyro_vectors
    })
 
    fea_df['Date'] = fea_df['Date'].astype(str).str.replace('_1', '', regex=False)
    fea_df = fea_df.drop_duplicates(subset='Date', ignore_index=True)
    fea_df["Date"] = pd.to_datetime(fea_df["Date"]).dt.strftime("%d.%m.%Y")
    fea_df = fea_df.sort_values(by="Date").reset_index(drop=True)
    data = pd.read_csv(f'awcr/{athlete_file}')
    datas = pd.merge(data, fea_df, on='Date', how='outer')
    output_file = os.path.join('timeseries', athlete_file)
    datas.to_csv(output_file, index=False)
    #break
            
            


In [26]:
fea_df = pd.DataFrame({
    "Date": dates,
    "Movement_Ratio": movement_ratio[1:],
    "Mean_HR": mean_hrs,
    "Max_HR": max_hrs,
    "Min_HR": min_hrs,
    "SDNN_HR": sdnn_hrs,
    "Total_Distance": total_distances,
    "Mean_Speed": mean_speeds,
    "Max_Speed": max_speeds,
    "Var_Speed": var_speeds,
    "Mean_Acceleration": mean_accelerations,
    "Max_Acceleration": max_accelerations,
    "Var_Acceleration": var_accelerations,
    "Shock_Count": shock_counts,
    "Mean_Gyro_Vector": mean_gyro_vectors,
    "Var_Gyro_Vector": var_gyro_vectors
})

fea_df['Date'] = fea_df['Date'].astype(str).str.replace('_1', '', regex=False)
fea_df = fea_df.drop_duplicates(subset='Date', ignore_index=True)
fea_df["Date"] = pd.to_datetime(fea_df["Date"]).dt.strftime("%d.%m.%Y")
fea_df = fea_df.sort_values(by="Date").reset_index(drop=True)
data = pd.read_csv(f'awcr/{athlete_file}')
datas = pd.merge(data, fea_df, on='Date', how='outer')
output_file = os.path.join('timeseries', athlete_file)
datas.to_csv(output_file, index=False)